# FastMRI 데이터 분석 및 검증

이 노트북은 `data1` 폴더에 있는 FastMRI 챌린지 데이터셋을 분석하고 검증하는 작업을 수행합니다.

**작업 목표:**
1.  **데이터 통계 분석**: 모든 k-space HDF5 파일을 순회하며 `slice`, `coil`, `H`, `W` 차원의 분포와 코일 개수의 통계를 분석합니다.
2.  **동일 스캔 검증**: `train` 데이터셋 내에서, 가속도만 다른 `_acc4` 파일과 `_acc8` 파일이 동일한 원본 k-space에서 왔는지 검증합니다.

## 1. 환경 설정 및 라이브러리 임포트

In [13]:
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.width', 100)

## 2. 데이터 통계 분석

모든 `kspace` 폴더를 순회하여 각 `.h5` 파일의 `kspace` 텐서 형태 정보를 수집하고 분석합니다.

In [14]:
base_dir = Path('data1')
kspace_dirs = [
    p for p in base_dir.rglob('*') 
    if p.is_dir() and p.name == 'kspace'
]

print(f"발견된 kspace 폴더들:\n{[str(d) for d in kspace_dirs]}")

Exception ignored in: <function tqdm.__del__ at 0x108e1a710>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/fmri/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/anaconda3/envs/fmri/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


발견된 kspace 폴더들:
['data1/leaderboard/acc4/kspace', 'data1/leaderboard/acc8/kspace', 'data1/train/kspace', 'data1/val/kspace']


In [15]:
all_files_info = []

for kspace_dir in kspace_dirs:
    h5_files = list(kspace_dir.glob('*.h5'))
    if not h5_files:
        continue
        
    for h5_file in tqdm(h5_files, desc=f"Scanning {kspace_dir.parent.name}/{kspace_dir.parent.parent.name}"):
        try:
            with h5py.File(h5_file, 'r') as f:
                if 'kspace' not in f:
                    continue
                slices, coils, H, W = f['kspace'].shape
                all_files_info.append({
                    'path': str(h5_file.parent.parent.parent.name),
                    'type': str(h5_file.parent.parent.name),
                    'filename': h5_file.name,
                    'slices': slices,
                    'coils': coils,
                    'H': H,
                    'W': W
                })
        except Exception as e:
            print(f"Error processing {h5_file}: {e}")

df_stats = pd.DataFrame(all_files_info)


Scanning acc4/leaderboard: 100%|██████████| 58/58 [00:00<00:00, 2330.64it/s]

Scanning acc8/leaderboard: 100%|██████████| 58/58 [00:00<00:00, 3355.30it/s]

Scanning train/data1: 100%|██████████| 340/340 [00:00<00:00, 4182.72it/s]

Scanning val/data1: 100%|██████████| 60/60 [00:00<00:00, 3881.04it/s]


### 2.1. 코일(Coil) 개수 분포

전체 데이터셋에서 코일 개수의 최솟값, 최댓값을 확인하고 각 코일 개수별로 몇 개의 파일이 있는지 집계합니다.

In [16]:
if not df_stats.empty:
    print("--- 코일 개수 통계 ---")
    print(f"최소 코일 개수: {df_stats['coils'].min()}")
    print(f"최대 코일 개수: {df_stats['coils'].max()}\n")
    
    coil_counts = df_stats['coils'].value_counts().sort_index().reset_index()
    coil_counts.columns = ['Coil 개수', '파일 수']
    print("--- 코일 개수별 파일 수 ---")
    print(coil_counts.to_string(index=False))
else:
    print("분석할 h5 파일을 찾지 못했습니다.")

--- 코일 개수 통계 ---
최소 코일 개수: 4
최대 코일 개수: 20

--- 코일 개수별 파일 수 ---
 Coil 개수  파일 수
       4    13
       8     2
      10     2
      12    24
      14    37
      15   258
      16    94
      18     3
      20    83


### 2.2. Slice, Height, Width 분포

k-space 텐서의 나머지 차원(slice, H, W)에 대한 기술 통계를 확인합니다.

In [17]:
if not df_stats.empty:
    print("--- Slice, Height, Width 기술 통계 ---")
    print(df_stats[['slices', 'H', 'W']].describe())
else:
    print("분석할 h5 파일을 찾지 못했습니다.")

--- Slice, Height, Width 기술 통계 ---
           slices           H           W
count  516.000000  516.000000  516.000000
mean    14.746124  704.000000  382.116279
std      2.398115   64.062106   15.630753
min     10.000000  640.000000  320.000000
25%     13.000000  640.000000  368.000000
50%     16.000000  704.000000  392.000000
75%     16.000000  768.000000  396.000000
max     22.000000  768.000000  480.000000


## 3. `acc4` vs `acc8` 스캔 검증

### 검증 방법론 제안
단순히 k-space 데이터를 모두 비교하는 것은 비효율적이므로, 다음과 같은 합리적인 두 단계 검증 방법을 사용합니다.

1.  **메타데이터(Metadata) 비교**: HDF5 파일의 `attrs`에 저장된 메타데이터를 비교합니다. 동일한 환자의 스캔이라면 `patient_id`와 같은 주요 속성들이 일치해야 합니다. 이 방법이 가장 빠르고 효율적입니다.

2.  **데이터 샘플링 비교**: 메타데이터 확인 후, 추가적인 검증을 위해 실제 k-space 데이터를 비교합니다. `acc4`와 `acc8` 마스크가 모두 1인, 즉 **공통으로 샘플링된 영역의 데이터는 완전히 일치**해야 합니다. 모든 슬라이스를 비교할 필요 없이, 중앙부의 대표 슬라이스 하나에서 공통 샘플링된 데이터가 일치하는지 확인하여 데이터 지문(fingerprint)처럼 사용합니다.

In [21]:
import h5py
import numpy as np
from pathlib import Path

def verify_scans_data_only(file_acc4: Path, file_acc8: Path) -> (bool, str):
    """
    메타데이터 없이 오직 데이터의 Shape과 샘플 값만으로
    두 h5 파일이 동일한 원본 스캔인지 검증합니다.
    """
    if not file_acc4.exists() or not file_acc8.exists():
        return False, "파일 없음"

    try:
        with h5py.File(file_acc4, 'r') as f4, h5py.File(file_acc8, 'r') as f8:
            kspace4 = f4['kspace']
            kspace8 = f8['kspace']

            # 1단계: k-space 텐서의 Shape 전체 비교
            # (slices, coils, H, W) 차원이 모두 동일해야 합니다.
            if kspace4.shape != kspace8.shape:
                return False, f"k-space Shape 불일치. F1:{kspace4.shape}, F2:{kspace8.shape}"

            # 2단계: 공통 샘플링 영역의 '전체' 데이터 비교
            mask4 = f4['mask'][:]
            mask8 = f8['mask'][:]
            
            # 두 마스크에서 공통으로 샘플링된 k-space 주파수 축(W)의 인덱스를 찾습니다.
            common_cols = np.where((mask4 == 1) & (mask8 == 1))[0]

            if len(common_cols) == 0:
                return False, "공통 샘플링 영역 없음"

            # '모든 슬라이스'와 '모든 코일'에 대해 공통 영역 데이터 추출
            # Ellipsis(...)는 모든 이전 차원(slice, coil, H)을 의미합니다.
            kspace4_common = kspace4[..., common_cols]
            kspace8_common = kspace8[..., common_cols]
            
            # 복소수 배열 전체를 비교합니다.
            if not np.allclose(kspace4_common, kspace8_common):
                return False, "k-space 샘플 데이터 불일치"

    except Exception as e:
        return False, f"파일 처리 중 오류: {e}"
    
    return True, "검증 성공 (Shape 및 k-space 샘플 데이터 일치)"

In [22]:
print("--- Train 데이터셋 acc4 vs acc8 검증 시작 ---\n")
train_kspace_dir = base_dir / 'train/kspace'
verification_results = []
all_verified = True

scan_types = {
    'brain': range(1, 86),
    'knee': range(1, 86)
}

for organ, indices in scan_types.items():
    print(f"[{organ.capitalize()} Scans 검증]")
    num_failed = 0
    for i in tqdm(indices, desc=f"Verifying {organ}"):
        file4 = train_kspace_dir / f"{organ}_acc4_{i}.h5"
        file8 = train_kspace_dir / f"{organ}_acc8_{i}.h5"
        
        # 수정된 함수 호출
        is_same, reason = verify_scans_data_only(file4, file8)
        result = {
            'organ': organ,
            'index': i,
            'verified': is_same,
            'reason': reason
        }
        verification_results.append(result)
        if not is_same:
            all_verified = False
            num_failed += 1
            print(f"  - 검증 실패: {organ}_acc*_{i}.h5, 이유: {reason}")
    
    if num_failed == 0:
        print(f"  >> 모든 {organ} 스캔 페어가 성공적으로 검증되었습니다.\n")
    else:
        print(f"  >> {organ} 스캔 중 {num_failed}개 페어가 검증에 실패했습니다.\n")

print("--- 검증 완료 ---")
if all_verified:
    print("✅ 모든 스캔 페어가 동일한 원본 데이터임을 성공적으로 확인했습니다.")
else:
    print("❌ 일부 스캔 페어에서 불일치가 발견되었습니다. 상세 내용은 위 로그를 확인하세요.")

--- Train 데이터셋 acc4 vs acc8 검증 시작 ---

[Brain Scans 검증]


  - 검증 실패: brain_acc*_1.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: brain_acc*_2.h5, 이유: k-space Shape 불일치. F1:(16, 8, 768, 396), F2:(16, 12, 768, 396)


  - 검증 실패: brain_acc*_3.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: brain_acc*_4.h5, 이유: k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 16, 768, 396)
  - 검증 실패: brain_acc*_5.h5, 이유: k-space Shape 불일치. F1:(16, 14, 768, 396), F2:(16, 16, 768, 396)
  - 검증 실패: brain_acc*_6.h5, 이유: k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 4, 768, 392)
  - 검증 실패: brain_acc*_7.h5, 이유: k-space Shape 불일치. F1:(16, 12, 768, 396), F2:(14, 14, 768, 396)
  - 검증 실패: brain_acc*_8.h5, 이유: k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 16, 768, 396)


  - 검증 실패: brain_acc*_9.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: brain_acc*_10.h5, 이유: k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(16, 20, 768, 396)
  - 검증 실패: brain_acc*_11.h5, 이유: k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 4, 768, 392)
  - 검증 실패: brain_acc*_12.h5, 이유: k-space Shape 불일치. F1:(16, 12, 768, 396), F2:(16, 14, 768, 396)
  - 검증 실패: brain_acc*_13.h5, 이유: k-space Shape 불일치. F1:(16, 12, 768, 396), F2:(16, 4, 768, 392)
  - 검증 실패: brain_acc*_14.h5, 이유: k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(16, 14, 768, 396)
  - 검증 실패: brain_acc*_15.h5, 이유: k-space Shape 불일치. F1:(16, 8, 768, 396), F2:(16, 12, 768, 396)
  - 검증 실패: brain_acc*_16.h5, 이유: k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 16, 768, 396)
  - 검증 실패: brain_acc*_17.h5, 이유: k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(16, 20, 768, 396)
  - 검증 실패: brain_acc*_18.h5, 이유: k-space Shape 불일치. F1:(16, 4, 768, 392), F2:(16, 16, 768, 396)


  - 검증 실패: brain_acc*_19.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: brain_acc*_20.h5, 이유: k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(16, 14, 768, 396)
  - 검증 실패: brain_acc*_21.h5, 이유: k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(12, 20, 768, 396)
  - 검증 실패: brain_acc*_22.h5, 이유: k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(16, 20, 768, 396)


...

  - 검증 실패: knee_acc*_34.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: knee_acc*_35.h5, 이유: k-space Shape 불일치. F1:(16, 15, 640, 372), F2:(14, 15, 640, 368)
  - 검증 실패: knee_acc*_36.h5, 이유: k-space Shape 불일치. F1:(15, 15, 640, 368), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_37.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 372), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_38.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(13, 15, 640, 372)
  - 검증 실패: knee_acc*_39.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 322), F2:(15, 15, 640, 386)
  - 검증 실패: knee_acc*_40.h5, 이유: k-space Shape 불일치. F1:(15, 15, 640, 368), F2:(14, 15, 640, 368)
  - 검증 실패: knee_acc*_41.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(20, 15, 640, 368)
  - 검증 실패: knee_acc*_42.h5, 이유: k-space Shape 불일치. F1:(10, 15, 640, 372), F2:(13, 15, 640, 372)
  - 검증 실패: knee_acc*_43.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(10, 15, 640, 372)
  - 검증 실패: knee_acc*_44.h5, 이유: k-space Shape 불일치. F1:(17, 15, 640, 388), F2:(13, 15,

  - 검증 실패: knee_acc*_48.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: knee_acc*_49.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 372), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_50.h5, 이유: k-space Shape 불일치. F1:(15, 15, 640, 368), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_51.h5, 이유: k-space Shape 불일치. F1:(10, 15, 640, 372), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_52.h5, 이유: k-space Shape 불일치. F1:(16, 15, 640, 368), F2:(12, 15, 640, 368)
  - 검증 실패: knee_acc*_53.h5, 이유: k-space Shape 불일치. F1:(18, 15, 640, 368), F2:(14, 15, 640, 386)
  - 검증 실패: knee_acc*_54.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 368), F2:(11, 15, 640, 372)
  - 검증 실패: knee_acc*_55.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(14, 15, 640, 368)
  - 검증 실패: knee_acc*_56.h5, 이유: k-space Shape 불일치. F1:(19, 15, 640, 480), F2:(12, 15, 640, 368)
  - 검증 실패: knee_acc*_57.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(12, 15, 640, 368)
  - 검증 실패: knee_acc*_58.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 372), F2:(12, 15,

  - 검증 실패: knee_acc*_59.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: knee_acc*_60.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 372), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_61.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 320), F2:(13, 15, 640, 372)
  - 검증 실패: knee_acc*_62.h5, 이유: k-space Shape 불일치. F1:(14, 15, 640, 368), F2:(17, 15, 640, 368)
  - 검증 실패: knee_acc*_63.h5, 이유: k-space Shape 불일치. F1:(10, 15, 640, 372), F2:(13, 15, 640, 372)
  - 검증 실패: knee_acc*_64.h5, 이유: k-space Shape 불일치. F1:(11, 15, 640, 372), F2:(13, 15, 640, 372)
  - 검증 실패: knee_acc*_65.h5, 이유: k-space Shape 불일치. F1:(16, 15, 640, 368), F2:(13, 15, 640, 368)
  - 검증 실패: knee_acc*_66.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 372), F2:(14, 15, 640, 368)



Verifying knee: 100%|██████████| 85/85 [00:01<00:00, 45.44it/s]

  - 검증 실패: knee_acc*_67.h5, 이유: k-space 샘플 데이터 불일치
  - 검증 실패: knee_acc*_68.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 372), F2:(10, 15, 640, 372)
  - 검증 실패: knee_acc*_69.h5, 이유: k-space Shape 불일치. F1:(16, 15, 640, 368), F2:(12, 15, 640, 372)
  - 검증 실패: knee_acc*_70.h5, 이유: k-space Shape 불일치. F1:(14, 15, 640, 368), F2:(12, 15, 640, 372)
  - 검증 실패: knee_acc*_71.h5, 이유: k-space Shape 불일치. F1:(10, 15, 640, 372), F2:(12, 15, 640, 372)
  - 검증 실패: knee_acc*_72.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(18, 15, 640, 368)
  - 검증 실패: knee_acc*_73.h5, 이유: k-space Shape 불일치. F1:(12, 15, 640, 368), F2:(14, 15, 640, 368)
  - 검증 실패: knee_acc*_74.h5, 이유: k-space Shape 불일치. F1:(11, 15, 640, 368), F2:(15, 15, 640, 368)
  - 검증 실패: knee_acc*_75.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 372), F2:(16, 15, 640, 368)
  - 검증 실패: knee_acc*_76.h5, 이유: k-space Shape 불일치. F1:(13, 15, 640, 368), F2:(14, 15, 640, 368)
  - 검증 실패: knee_acc*_77.h5, 이유: k-space Shape 불일치. F1:(14, 15, 640, 372), F2:(13, 15,

### 3.1. 검증 실패 상세 목록 (해당하는 경우)

만약 검증에 실패한 파일이 있다면 아래에 목록을 출력합니다.

In [23]:
df_results = pd.DataFrame(verification_results)
failed_scans = df_results[df_results['verified'] == False]

if not failed_scans.empty:
    print("--- 검증 실패 목록 ---")
    print(failed_scans.to_string(index=False))
else:
    print("모든 페어 검증에 성공하여 실패 목록이 없습니다.")

--- 검증 실패 목록 ---
organ  index  verified                                                          reason
brain      1     False                                              k-space 샘플 데이터 불일치
brain      2     False  k-space Shape 불일치. F1:(16, 8, 768, 396), F2:(16, 12, 768, 396)
brain      3     False                                              k-space 샘플 데이터 불일치
brain      4     False k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 16, 768, 396)
brain      5     False k-space Shape 불일치. F1:(16, 14, 768, 396), F2:(16, 16, 768, 396)
brain      6     False  k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 4, 768, 392)
brain      7     False k-space Shape 불일치. F1:(16, 12, 768, 396), F2:(14, 14, 768, 396)
brain      8     False k-space Shape 불일치. F1:(16, 20, 768, 396), F2:(16, 16, 768, 396)
brain      9     False                                              k-space 샘플 데이터 불일치
brain     10     False k-space Shape 불일치. F1:(16, 16, 768, 396), F2:(16, 20, 768, 396)
brain     11     False  k-